# Pipeline 1

In [1]:
from pipeline import SerialAnnotator, T5Converter
import time
from pprint import pprint

main_virtuoso_noreranking


In [2]:
annotator = SerialAnnotator()
converter = T5Converter()

In [3]:
def pipe_batch(linked, wikisparqls):
  batched = []
  s = time.time()
  annotated = annotator.batch_annotate(linked)
  e = time.time()
  assert len(linked) == len(annotated)
  annotator_time = e - s
  converter_time = 0
  for i, single_annotated in enumerate(annotated):
    # annotated = annotator.annotate(**single_linked)
    # try:
    #   annotator_time += e - s
    # except Exception as e:
    #   print("[Annotator Error]:", e)
    #   continue
    s = time.time()
    try:
      converted = converter.preprocess(**single_annotated, wikisparql=wikisparqls[i])
    except Exception as e:
      print("[Converter Error]:", e)
      continue
      
    e = time.time()
    converter_time += e - s
    batched.append([linked[i], annotated[i], converted])
  print("Anno batch time:", annotator_time)
  print("Conv batch time:", converter_time)
  return batched

## Data

In [4]:
from pathlib import Path
from pprint import pprint
import pandas as pd
import json

# data_path = Path("..") / "t5-for-sparql" / "data" / "lcquad2" / "train.json"
data_path = Path("..") / "t5-for-sparql" / "falcon_links" / "top1ents" / "link_24066.json"

In [5]:
with open(data_path) as f:
  data_json = json.load(f)
sample = data_json[1]

In [6]:
import re

def retrieve_gold_links(masked_wikisparql):
  fragments = masked_wikisparql.upper().strip().split(' ')
  ents = []
  rels = []
  result_dict = {
    'ents': ents,
    'rels': rels,
  }
  for fragment in fragments:
    id_match = re.search('[QP][1-9]+',fragment)
    if id_match:
      id_raw = id_match.group(0)
      if id_raw[0] == 'Q':
        prefix = 'wdt:'
        uri = 'http://www.wikidata.org/entity/' + id_raw
        target_arr = ents
      elif id_raw[0] == 'P':
        prefix = 'wd:'
        uri =  'http://www.wikidata.org/prop/direct/' + id_raw
        target_arr = rels
      else:
        raise ValueError('ID does not start with P or Q')
      match_dict = {
        'id' : id_raw,
        'prefix' : prefix,
        'uri' : uri,
      }
      target_arr.append(match_dict)
  return result_dict

In [7]:
for entry in data_json:
  old_linked, _, old_converted = entry
  wikisparql = old_converted['labels']
  gold_links = retrieve_gold_links(wikisparql)
  entry.append(gold_links)

In [8]:
data_json[0]

[{'utterance': 'What is Delta Air Lines periodical literature mouthpiece?',
  'ents': [{'uri': 'http://www.wikidata.org/entity/Q188920',
    'prefix': 'wd:',
    'id': 'Q188920'}],
  'rels': []},
 {'utterance': 'What is Delta Air Lines periodical literature mouthpiece?',
  'fragments': ['[DEF]', 'wd:', 'Q188920 Delta']},
 {'inputs': 'What is Delta Air Lines periodical literature mouthpiece? <extra_id_59> <extra_id_53> Q188920 Delta',
  'labels': '<extra_id_6> <extra_id_21> <extra_id_39> <extra_id_19> <extra_id_33> <extra_id_53> q188920 <extra_id_54> p2813 <extra_id_39> <extra_id_38> <extra_id_39> <extra_id_54> p31 <extra_id_53> q1002697 <extra_id_15>'},
 {'ents': [{'id': 'Q18892',
    'prefix': 'wdt:',
    'uri': 'http://www.wikidata.org/entity/Q18892'},
   {'id': 'Q1', 'prefix': 'wdt:', 'uri': 'http://www.wikidata.org/entity/Q1'}],
  'rels': [{'id': 'P2813',
    'prefix': 'wd:',
    'uri': 'http://www.wikidata.org/prop/direct/P2813'},
   {'id': 'P31',
    'prefix': 'wd:',
    'uri': '

In [9]:
responses = []
batch_new_linked = []
batch_ans = []
last = 0
cutoff = 0
total_len = len(data_json)
for i, data in enumerate(data_json):
    linked, annotated, converted, gold_linked = data
    if i < cutoff:
        continue
    question = linked['utterance']
    new_ents = linked['ents']
    old_ents_set = set(x['id'] for x in linked['ents'])
    for gold_ent in gold_linked['ents']:
        if gold_ent['id'] not in old_ents_set:
            new_ents.append(gold_ent)
    new_linked = {
        'utterance': question,
        'ents': new_ents,
        'rels':[],
    }
    answer = converted['labels']
    batch_new_linked.append(new_linked)
    batch_ans.append(answer)
    if ((i + 1) % 50) == 0 or i == total_len - 1:
        print("[Pipeline2]:", f"Processing {last}-{i}")
        try:
            for linked, annotated, converted in pipe_batch(batch_new_linked, batch_ans):
                responses.append([linked, annotated, converted])
            batch_new_linked = []
            batch_ans = []
            last = i
        except Exception as e:
            raise e
            # print(e)
            # continue
        with open(f"../t5-for-sparql/falcon_links/5ents-gold_0rels/link_{i}.json", "w") as f:
            json.dump(responses, f, indent=2, separators=(',',':'))

[Pipeline2]: Processing 0-49


Anno batch time: 25.408459663391113
Conv batch time: 0.0005373954772949219
[Pipeline2]: Processing 49-99


Anno batch time: 23.667537927627563
Conv batch time: 0.0004725456237792969
[Pipeline2]: Processing 99-149


Anno batch time: 28.38252878189087
Conv batch time: 0.00045037269592285156
[Pipeline2]: Processing 149-199


Anno batch time: 23.77673602104187
Conv batch time: 0.0004849433898925781
[Pipeline2]: Processing 199-249


...

Anno batch time: 22.454195499420166
Conv batch time: 0.0004000663757324219


[Pipeline2]: Processing 23999-24049


Anno batch time: 47.50054955482483
Conv batch time: 0.0004074573516845703


[Pipeline2]: Processing 24049-24063


Anno batch time: 16.93176555633545
Conv batch time: 0.0001537799835205078
